Importação das bibliotecas necessarias

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score, classification_report

Carregamento do Dataset

In [6]:
data = pd.read_csv('/content/creditcard.csv')

Pré-processamento dos dados

In [7]:
X = data.drop('Class', axis=1)
y = data['Class']

#divisão de dados de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#normalização com StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Treinamento do modelo

In [ ]:
modelo = MLPClassifier(random_state=42)
modelo.fit(X_train, y_train)

Score do modelo

In [ ]:
y_pred = modelo.predict(X_test)

acuracia = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, modelo.predict_proba(X_test)[:, 1])

print(f'Acurácia: {acuracia}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')
print(f'AUC-ROC: {auc_roc}')

Relatório de classificação do modelo

In [ ]:
print(classification_report(y_test, y_pred))

Definição de uma faixa de valores para os hiperparâmetros a serem otimizados

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Definindo a faixa de valores para os hiperparâmetros
parametros = {
    'hidden_layer_sizes': [(50,), (100,), (50,50)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive'],
}

Aplicação dos modelos de Grid Search e do Random Search

In [ ]:
#Aplicação do modelo de Grid Search
grid_search = GridSearchCV(estimator=MLPClassifier(random_state=42),
                           param_grid=parametros,
                           scoring='f1',
                           cv=3,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)

print("Melhores parâmetros do Grid Search:")
print(grid_search.best_params_)
print("Melhor F1-score do Grid Search:")
print(grid_search.best_score_)

In [ ]:
#Aplicação do modelo de Random Search
random_search = RandomizedSearchCV(estimator=MLPClassifier(random_state=42),
                                   param_distributions=parametros,
                                   n_iter=10,  # número de combinações aleatórias
                                   scoring='f1',
                                   cv=3,
                                   n_jobs=-1,
                                   random_state=42)

random_search.fit(X_train, y_train)

print("Melhores parâmetros do Random Search:")
print(random_search.best_params_)
print("Melhor F1-score do Random Search:")
print(random_search.best_score_)

Score do modelo otimizado, apos encontrar os melhores hiperparâmetros

In [ ]:
melhor_modelo = grid_search.best_estimator_  # random_search.best_estimator_
y_pred_otimizado = melhor_modelo.predict(X_test)

acuracia_otimizada = accuracy_score(y_test, y_pred_otimizado)
recall_otimizado = recall_score(y_test, y_pred_otimizado)
f1_otimizado = f1_score(y_test, y_pred_otimizado)
auc_roc_otimizado = roc_auc_score(y_test, melhor_modelo.predict_proba(X_test)[:, 1])

#Resultado do Modelo otimizado
print(f'Acurácia Otimizada: {acuracia_otimizada}')
print(f'Recall Otimizado: {recall_otimizado}')
print(f'F1-score Otimizado: {f1_otimizado}')
print(f'AUC-ROC Otimizado: {auc_roc_otimizado}')

Comparação dos resutados obtidos com o modelo original

In [ ]:
# Resultados do modelo original
print("Resultados do Modelo Original:")
print(f"Acurácia: {acuracia}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print(f"AUC-ROC: {auc_roc}")

# Resultados do modelo otimizado
print("\nResultados do Modelo Otimizado:")
print(f"Acurácia Otimizada: {acuracia_otimizada}")
print(f"Recall Otimizado: {recall_otimizado}")
print(f"F1-score Otimizado: {f1_otimizado}")
print(f"AUC-ROC Otimizado: {auc_roc_otimizado}")

# **Análise do Impacto dos Hiperparâmetros**

###Modelo Original:

*   Acurácia: 99.95%
*   Recall: 82.65%
*   F1-score: 85.26%
*   AUC-ROC: 97.42%


### Modelo Otimizado:

*   Acurácia Otimizada: 99.95%
*   Recall Otimizado: 76.53%
*   F1-score Otimizado: 85.23%
*   AUC-ROC Otimizado: 98.13%

### Modelo Otimizado com Grid Search
**Melhores Parâmetros:**

activation: 'tanh'

# *   alpha: 0.0001
*   hidden_layer_sizes: (50,)
*   learning_rate: 'constant'
*   solver: 'adam'

F1-score do Grid Search: 84.34%

### Modelo Otimizado com Random Search
**Melhores Parâmetros:**

activation: 'tanh'

*   solver: 'sgd'
*   learning_rate: 'constant'
*   hidden_layer_sizes: (100,)
*   alpha: 0.01
*   activation: 'tanh'

F1-score do Random Search: 82.54%



---

### Impacto nas Métricas
 - **Acurácia**: Houve uma ligeira melhora, quase que impercéptivel, entretanto a taxa já era alta inicialmente.
 - **Recall**: Houve uma queda significativa, indicando uma menor detecção de fraudes.
 - **F1-score**: Permaneceu praticamente estável, mostrando equilíbrio entre precisão e recall.
 - **AUC-ROC**: Melhorou parcialmente, indicando uma melhor capacidade de discriminação entre classes.

#**Observações**
**Melhorias**: O AUC-ROC melhorou, mas o recall diminuiu, o que pode ser problemático para detecção de fraudes.

**Desempenho Geral:** A manutenção do F1-score indica equilíbrio, mas o recall reduzido sugere que o modelo pode estar menos sensível a fraudes.

**Limitações e Próximos Passos:** Considerar
balanceamento de classes ou ajustes adicionais nos hiperparâmetros para melhorar o recall.